# The ML Modeling Process Basics 
In this notebook, we will go through some of the basic techinques for modeling data. This is a companion workbook for the 365 Data Science course on ML Process. This notebook only foucses on implementation. Check out the course or the documentation for the in-depth explanations of each step

In this case, we will be trying to predict if we can predict a stroke from the above dataset. 

We will cover:
- Baseline creation
- Model selection
- Parameter tuning
     - manual
     - gridsearch
     - random search
     - basian optomization
- Ensemble models

### On the Data 
This dataset is a good representation of real world data that can have valuable impact when analyzed. We will be exploring the accuracy of different models for predicting if someone will purchase an auto insurance policy or not. We will first lightly explore the data, create our train, test / validation sets, then we will ceate a baseline model. To get the best results we will compare other algorithms to our basline and use various parameter tuning techniques to see which model produces the best results. At the end we will explore some ensemble models to see what produces the best results. 

The focus of this notebook is the modeling process. If you're interested in the specifics of differen machine learning algorithms, check out our other course specifically on that. 

In [1]:
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_csv('./data/Cross_Sell_Prediction.csv')

In [3]:
#look at basic data for continuous variables 
df.describe()

,id,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000
mean,190555.000000,38.822584,0.997869,26.388807,0.458210,30564.389581,112.034295,154.347397,0.122563
std,110016.836208,15.511611,0.046110,13.229888,0.498251,17213.155057,54.203995,83.671304,0.327936
min,1.000000,20.000000,0.000000,0.000000,0.000000,2630.000000,1.000000,10.000000,0.000000
25%,95278.000000,25.000000,1.000000,15.000000,0.000000,24405.000000,29.000000,82.000000,0.000000
50%,190555.000000,36.000000,1.000000,28.000000,0.000000,31669.000000,133.000000,154.000000,0.000000
75%,285832.000000,49.000000,1.000000,35.000000,1.000000,39400.000000,152.000000,227.000000,0.000000
max,381109.000000,85.000000,1.000000,52.000000,1.000000,540165.000000,163.000000,299.000000,1.000000


In [4]:
df.describe(include=object)

,Gender,Vehicle_Age,Vehicle_Damage
count,381109,381109,381109
unique,2,3,2
top,Male,1-2 Year,Yes
freq,206089,200316,192413


In [5]:
#small enough number of null values we will just remove them.
df.isnull().sum()

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
Response                0
dtype: int64

In [6]:
# check for possible nulls in categoricals / non answers 
for i in df.select_dtypes(include=['object']).columns:
    print(df[i].value_counts())

Gender
Male      206089
Female    175020
Name: count, dtype: int64
Vehicle_Age
1-2 Year     200316
< 1 Year     164786
> 2 Years     16007
Name: count, dtype: int64
Vehicle_Damage
Yes    192413
No     188696
Name: count, dtype: int64


In [7]:
#dropping variable for now, but could likely be used to imrove our models with some engineering! 
df.Policy_Sales_Channel.value_counts()

Policy_Sales_Channel
152.0    134784
26.0      79700
124.0     73995
160.0     21779
156.0     10661
          ...  
149.0         1
43.0          1
144.0         1
143.0         1
41.0          1
Name: count, Length: 155, dtype: int64

In [8]:
df_trimmed = df.loc[:,['Gender','Age','Driving_License','Previously_Insured','Vehicle_Age','Vehicle_Damage','Annual_Premium','Vintage','Response']]

In [9]:
#drop null values and create dummy variables
df_final = pd.get_dummies(df_trimmed).dropna()

In [10]:
df_final.columns

Index(['Age', 'Driving_License', 'Previously_Insured', 'Annual_Premium',
       'Vintage', 'Response', 'Gender_Female', 'Gender_Male',
       'Vehicle_Age_1-2 Year', 'Vehicle_Age_< 1 Year', 'Vehicle_Age_> 2 Years',
       'Vehicle_Damage_No', 'Vehicle_Damage_Yes'],
      dtype='object')

In [11]:
df_final.Response.value_counts()

Response
0    334399
1     46710
Name: count, dtype: int64

In [12]:
#Create train test split 

from sklearn.model_selection import train_test_split
X = df_final.drop('Response', axis =1)
y = df_final.loc[:,['Response']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
#balance the data (SMOTE) Try this if interested: https://www.kaggle.com/code/kenjee/dealing-with-imbalanced-data-section-10
"""from imblearn.over_sampling import SMOTE 
smote = SMOTE(sampling_strategy =1)

X_train, y_train = smote.fit_resample(X_train,y_train)"""

'from imblearn.over_sampling import SMOTE \nsmote = SMOTE(sampling_strategy =1)\n\nX_train, y_train = smote.fit_resample(X_train,y_train)'

# Creating a Basline Model
How can we tell if our machine learning models are any good? To evaluate performance, we need to benchmark against something. In this case, we will create two baslines for our model. First, we can simply look at the average of our data for a numeric value. If we were going to predict the age, we could simply guess the average age for every candidate. 

On the other hand, for a categorical variable, we could simply guess 50/50 or the ratio of the categories in the data. In this case, the conversion data is imbalanced with 46710/ 334399 samples being of the non-stroke cateogry. That means that if we guessed that everyone in the sample didn't have a stroke, we would have a 86.0% success rate. Since this data is slightly imblanaced, this would not be a good baseline for our model.

One of the most important steps that we need to take is choosing a good evluation metric. The notebook that covers specific evaluation metrics can be located here: 

Accuracy does not make sense because of the imbalanced nature of the data. For this example we will use F1 score as our model evaluation metric.

- F1 is calculated by 2*((precision*recall)/(precision+recall))

- Instead of a simple accuracy calculation which would give us a baseline of 96.1%, F1 score gives us an undefined number since both the precision and recall of a model that only predicted negatives would equal 0. 

- In this case, we want to use a simple basleline model like Naive Bayes to set our baseline based off of f1 score. You can use most models to create a baseline, but I like Naive bayes because it is quick and doesn't require much parameter tuning. (Full breakdown of Naieve Bayes in or Algorithms Course)

In [14]:
#import cross validation score
from sklearn.model_selection import cross_val_score

#import Naive Bayes Classifier 
from sklearn.naive_bayes import GaussianNB

#create classifier object
nb = GaussianNB()

#run cv for NB classifier
from sklearn.metrics import classification_report

nb_accuracy = cross_val_score(nb,X_train,y_train.values.ravel(), cv=3, scoring ='accuracy')
nb_f1 = cross_val_score(nb,X_train,y_train.values.ravel(), cv=3, scoring ='f1')

print('nb_accuracy: ' +str(nb_accuracy))
print('nb F1_Macro Score: '+str(nb_f1))
print('nb_accuracy_avg: ' + str(nb_accuracy.mean()) +'  |  lr_f1_avg: '+str(nb_f1.mean()))


#With these F1 scores, we can begin evaluating our model. While the accuracy is lower than if we only predicted 0 every time,
# our f1 score suggests we are doing a far better job of predicting stroke outcomes. 

nb_accuracy: [0.74467535 0.74676413 0.7414113 ]
nb F1_Macro Score: [0.40969243 0.41583439 0.41445341]
nb_accuracy_avg: 0.7442835922580899  |  lr_f1_avg: 0.4133267459130525


# Model Comparison & Selection 
After we have a baseline model to compare against, we want to evaluate how other models might perform on the same data. I like to experiment with other basic models with very little paramater tuning to see what performs well. This isn't an exact science and many people may do this step differently. After we set up the models, we can begin experimenting with parameter tuning. I find that model selection and parameter tuning is often an iterative process. For an analysis like this, trying different models, changing parameters, and experimenting with new engineered features is where I find myself spending most of my time working. 

In this section we will try:
- Logistic regression
- Decision Tree
- K Nearest Neighbors (KNN)

In [15]:
#Let's now experiment with a few different basic models 

## Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state =32)

dt_accuracy = cross_val_score(dt,X_train,y_train.values.ravel(), cv=3, scoring ='accuracy')
dt_f1 = cross_val_score(dt,X_train,y_train.values.ravel(), cv=3, scoring ='f1')

print('dt_accuracy: ' +str(dt_accuracy))
print('dt F1_Macro Score: '+str(dt_f1))
print('dt_accuracy_avg: ' + str(dt_accuracy.mean()) +'  |  dt_f1_avg: '+str(dt_f1.mean())+'\n')


## Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=32, max_iter = 2000, class_weight = 'balanced')

lr_accuracy = cross_val_score(lr,X_train,y_train.values.ravel(), cv=3, scoring ='accuracy')
lr_f1 = cross_val_score(lr,X_train,y_train.values.ravel(), cv=3, scoring ='f1')

print('lr_accuracy: ' +str(lr_accuracy))
print('lr F1_Macro Score: '+str(lr_f1))
print('lr_accuracy_avg: ' + str(lr_accuracy.mean()) +'  |  lr_f1_avg: '+str(lr_f1.mean())+'\n')


## KNN 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline

knn = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=3))
knn_accuracy = cross_val_score(knn,X_train,y_train.values.ravel(), cv=3, scoring ='accuracy')
knn_f1 = cross_val_score(knn,X_train,y_train.values.ravel(), cv=3, scoring ='f1')

print('knn_accuracy: ' +str(knn_accuracy))
print('knn F1_Macro Score: '+str(knn_f1))
print('knn_accuracy_avg: ' + str(knn_accuracy.mean()) +'  |  knn_f1_avg: '+str(knn_f1.mean()))


dt_accuracy: [0.82098599 0.82362665 0.82037672]
dt F1_Macro Score: [0.28017183 0.28002203 0.27523935]
dt_accuracy_avg: 0.8216631206597756  |  dt_f1_avg: 0.27847773734127484



/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sci

lr_accuracy: [0.63619189 0.64043857 0.63616531]
lr F1_Macro Score: [0.39470139 0.39871371 0.39484513]
lr_accuracy_avg: 0.637598589853285  |  lr_f1_avg: 0.39608674229036517

knn_accuracy: [0.84465736 0.84592634 0.84531909]
knn F1_Macro Score: [0.2362048  0.24182392 0.2459295 ]
knn_accuracy_avg: 0.8453009290324972  |  knn_f1_avg: 0.2413194051813984


# Model Comparison 
It looks like we chose a pretty good baseline. While it slightly underperforms all of our new models in accuracy, it outperforms all of them in F1 score which is what we care about most for this analysis. Let's look at how everything stacks up. 

|Model          | F1 Score      |
| :------------ | :-----------: |
| **Baseline Naive Bayes**  | **41.3%**     |
| Logistic Regression  | **35.0%**     |
| Decision Tree  | **27.6%**     |
| K Nearest Neighbors | **24.6%**     |



While all of our models outperformed our basline, we still can do better. We can now parameter tune! That means that we make adjustments to the model parameter inputs to better compensate for our specific data. One of the drawbacks of Naive Bayes is that it has virtually no paramaters that we can tune, so our inital results are about the best we will get with it without making changes to our data. 

# Manual Parameter Tuning
Let's try to do some parameter tuning with a few of these models:

Let's start with K Nearest Neighbors,which has a few parameters we can adjust, one of them being the number of K. K is how many other datapoints it uses to make its classification. If k= 3 it uses it sees what the samples 3 closest neighbors is and classifies it as the most common one. If k = 5, it uses its 5 closest datapoints. Let's change the number of k and see if that changes our results. 

In [16]:
#Knn Model Comparison 

#here we will loop through and see which value of k performs the best. 

for i in range(1,7):
    knn = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=i))
    knn_f1 = cross_val_score(knn,X_train,y_train.values.ravel(), cv=3, scoring ='f1')
    print('K ='+(str(i)) + (': ') + str(knn_f1.mean()))

#What we find is that k=1 is the best estimator for this specific model. We go from 24.6% to 27.6%, a decent improvement! 
#We also realize that KNN may not be the best approach here because of the imbalanced data. 
#The larger the K is, the more of the majority class will automatically be included.

K =1: 0.2759257833619041
K =2: 0.1292229598422949
K =3: 0.2413194051813984
K =4: 0.12708959225119731
K =5: 0.20576233671467703
K =6: 0.11913837004323076


# Randomized Parameter Tuning
Since KNN may not be the best choice, let's explore the deicision tree. Decision trees have a lot more features we can tune. We can tweak the following:
- criterion {gini, entropy, log loss}
- splitter {best, random}
- max depth {int, None}
- min_samples_split {int, None}
- min_samples_leave {int, None}
- min_weight_fraction_leaf {float}
- max_features {int, auto, sqrt, log2, None}
- max_leaf_nodes {int, None}
- min_impurity_decrease {float}
- class_weight {dict, balanced, None}
- ccp_alpha {float}

There are a lot of parameters to tune! If there are just 2 options for each one that would be 2^11, which is 2048 total configurations. In theory, there are infinate numbers of paramater configurations. How do we even get close to finding the best one? 

The answer here is randomized search. We through in all the parameters that we are interested in searching, and the model will randomly select a subset and return the one that produces the best results. 

Still, let's manually select a few paramaters we want to evaluate on and then use randomized search:
- criterion
- split strategy
- max depth
- min_samples_split
- max features

In [17]:
from sklearn.model_selection import RandomizedSearchCV

dt = DecisionTreeClassifier(random_state = 42)

features = {'criterion': ['gini','entropy'],
            'splitter': ['best','random'],
           'max_depth': [2,5,10,20,40,None],
           'min_samples_split': [2,5,10,15],
           'max_features': ['auto','sqrt','log2',None]}

rs_dt = RandomizedSearchCV(estimator = dt, param_distributions =features, n_iter =100, cv = 3, random_state = 42, scoring ='f1')

rs_dt.fit(X_train,y_train)

/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
81 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
81 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python3.9/site-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python

RandomizedSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [2, 5, 10, 20, 40, None],
                                        'max_features': ['auto', 'sqrt', 'log2',
                                                         None],
                                        'min_samples_split': [2, 5, 10, 15],
                                        'splitter': ['best', 'random']},
                   random_state=42, scoring='f1')

In [18]:
print('best stcore = ' + str(rs_dt.best_score_))
print('best params = ' + str(rs_dt.best_params_))

best stcore = 0.2804199162062474
best params = {'splitter': 'random', 'min_samples_split': 2, 'max_features': None, 'max_depth': 40, 'criterion': 'entropy'}


# GridsearchCV (Exhaustive Parameter Tuning)
With this we have improved our model f1 score from **27.6% to 27.9%**. This is a decent increase! We also narrowed down some of the features that produced good results. We may want to try a more exhaustive search this time. Gridsearch goes through all of the possible combinations within an range and returns the best outcome. 

This time, let's do an exhaustive search of a smaller number of features and see if we can improve our results even more. 

In [19]:
from sklearn.model_selection import GridSearchCV


features_gs = {'criterion': ['entropy'],
            'splitter': ['random'],
           'max_depth': np.arange(30,50,1), #getting more precise within range
           'min_samples_split': [2,3,4,5,6,7,8,9],
           'max_features': [None]}

gs_dt = GridSearchCV(estimator = dt, param_grid =features_gs, cv = 3, scoring ='f1') #we don't need random state because there isn't randomization like before

gs_dt.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'criterion': ['entropy'],
                         'max_depth': array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46,
       47, 48, 49]),
                         'max_features': [None],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9],
                         'splitter': ['random']},
             scoring='f1')

In [20]:
print('best stcore = ' + str(gs_dt.best_score_))
print('best params = ' + str(gs_dt.best_params_))

#looks like we can  do a little better with this gridsearch! 

best stcore = 0.2812318250838347
best params = {'criterion': 'entropy', 'max_depth': 45, 'max_features': None, 'min_samples_split': 2, 'splitter': 'random'}


# Bayesian Optimization
I wonnder if we can do better than the funnel approach that we took with random search and gridsearch. What if we used a slightly smarter algorithm to help evaluate our features. Maybe we could explore all of the variables from the previous examples and see if our model missed something. This is where Bayesian Optimization comes in. This is an iterative process where our model improves its understandings of the feature inputs as it goes. (Full breakdown in the video portion of the course)

Now let's try to use this with a larger feature set on the same classifier. This won't guarantee a better result as it still is not an exahustive search, but in theory it let's us cover ground in a more efficient way. 

In [21]:
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold

In [22]:

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import StratifiedKFold
# Choose cross validation method 
cv = StratifiedKFold(n_splits = 3)


bs_lr = BayesSearchCV(
    dt,
    {'criterion': Categorical(['gini','entropy']),
            'splitter': Categorical(['best','random']),
           'max_depth': Integer(10,50),
           'min_samples_split': Integer(2,15),
           'max_features': Categorical(['sqrt', 'log2', None])},
    random_state=42,
    n_iter= 100,
    cv= cv,
    scoring ='f1')
bs_lr.fit(X_train,y_train.values.ravel())

/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point ['gini', 50, 'log2', 2, 'random'] before, using random point ['entropy', 26, 'sqrt', 3, 'random']
  warnings.warn(
/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point ['gini', 50, 'log2', 2, 'random'] before, using random point ['entropy', 18, None, 13, 'best']
  warnings.warn(
/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point ['entropy', 50, None, 2, 'random'] before, using random point ['entropy', 46, None, 14, 'random']
  warnings.warn(
/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has be

BayesSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
              estimator=DecisionTreeClassifier(random_state=42), n_iter=100,
              random_state=42, scoring='f1',
              search_spaces={'criterion': Categorical(categories=('gini', 'entropy'), prior=None),
                             'max_depth': Integer(low=10, high=50, prior='uniform', transform='normalize'),
                             'max_features': Categorical(categories=('sqrt', 'log2', None), prior=None),
                             'min_samples_split': Integer(low=2, high=15, prior='uniform', transform='normalize'),
                             'splitter': Categorical(categories=('best', 'random'), prior=None)})

In [23]:
print('best stcore = ' + str(bs_lr.best_score_))
print('best params = ' + str(bs_lr.best_params_))

#while this didn't outperform our gridsearch, it is still a good approach to try when dealing with many different feature options. 
#it still did outperform our originial random search.

best stcore = 0.2812318250838347
best params = OrderedDict([('criterion', 'entropy'), ('max_depth', 46), ('max_features', None), ('min_samples_split', 2), ('splitter', 'random')])


# Selecting a Model
We still haven't been able to do better than our baseline. In most cases, we to tune multiple different models until we reach one that performs the best based on our evaluation criteria. We also want to use other considerations like training time, prediction time, prediction time or interperetability to select selct the best model for our use case. 

Since we have one tuned model, lets see if we can improve it by combining it with a few of the other models we have used. This process is called ensembling. In the case of classification, we often use a popular vote metric to select the best model. 

Let's see if an ensemble model of these three classifiers outperforms our baseline model. 

In [24]:
from sklearn.ensemble import VotingClassifier

dt_voting = DecisionTreeClassifier(**{'criterion': 'entropy', 'max_depth': 44, 'max_features': None, 'min_samples_split': 2, 'splitter': 'random'}) # ** allows you to pass in parameters as dict
knn_voting = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=1))
lr_voting = LogisticRegression(random_state=32, max_iter = 2000, class_weight = 'balanced')

ens = VotingClassifier(estimators = [('dt', dt_voting), ('knn', knn_voting), ('lr',lr_voting)], voting = 'hard')


In [25]:
voting_accuracy = cross_val_score(ens,X_train,y_train.values.ravel(), cv=3, scoring ='accuracy')
voting_f1 = cross_val_score(ens,X_train,y_train.values.ravel(), cv=3, scoring ='f1')

print('voting_accuracy: ' +str(voting_accuracy))
print('voting F1_Macro Score: '+str(voting_f1))
print('voting_accuracy_avg: ' + str(voting_accuracy.mean()) +'  |  voting_f1_avg: '+str(voting_f1.mean()))

/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sci

voting_accuracy: [0.79909138 0.80134945 0.79942648]
voting F1_Macro Score: [0.3251187  0.33286819 0.32567395]
voting_accuracy_avg: 0.7999557713752062  |  voting_f1_avg: 0.32788694537472346


In [26]:
ens = VotingClassifier(estimators = [('dt', dt_voting), ('knn', knn_voting), ('lr',lr_voting)], voting = 'soft')
voting_accuracy = cross_val_score(ens,X_train,y_train.values.ravel(), cv=3, scoring ='accuracy')
voting_f1 = cross_val_score(ens,X_train,y_train.values.ravel(), cv=3, scoring ='f1')

print('voting_accuracy: ' +str(voting_accuracy))
print('voting F1_Macro Score: '+str(voting_f1))
print('voting_accuracy_avg: ' + str(voting_accuracy.mean()) +'  |  voting_f1_avg: '+str(voting_f1.mean()))

/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sci

voting_accuracy: [0.79931629 0.80077593 0.79858308]
voting F1_Macro Score: [0.3257513  0.3313668  0.33172715]
voting_accuracy_avg: 0.7995584319509494  |  voting_f1_avg: 0.3296150837285653


# Stacked classifier 
In the case of the voting classifer, we didn't get better performance than our baseline model. Let's now try another type of ensembling called stacking. With stacking, we use the outputs of each of our individual models as features into a new model. In this case, where we have a decision tree, a naive baayes classifier, and a svc classifier, these will be the three features that a new model predicts on. 

Let's try running these three through a Naive Bayes Classifier and see what the results look like. 

In [27]:
from sklearn.ensemble import StackingClassifier

ens_stack = StackingClassifier(estimators = [('dt', dt_voting), ('lr',lr_voting), ('nb',GaussianNB())], final_estimator = GaussianNB())

In [28]:
stack_accuracy = cross_val_score(ens_stack,X_train,y_train.values.ravel(), cv=3, scoring ='accuracy')
stack_f1 = cross_val_score(ens_stack,X_train,y_train.values.ravel(), cv=3, scoring ='f1')

print('stacking_accuracy: ' +str(stack_accuracy))
print('stacking F1_Macro Score: '+str(stack_f1))
print('stacking_accuracy_avg: ' + str(stack_accuracy.mean()) +'  |  stack_f1_avg: '+str(stack_f1.mean()))

#in this case it didn't outperfrom, but it often does.

/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sci

stacking_accuracy: [0.73484695 0.7356199  0.72996345]
stacking F1_Macro Score: [0.42420092 0.43097038 0.42572763]
stacking_accuracy_avg: 0.733476769386369  |  stack_f1_avg: 0.4269663108037833


# Ensemble Models
The last main type of ensemble approach that we see is one that is designed that way algorithmically. Typically, random forest or gradient boosted models have ensembling built into their implementation. Let's explor random forest and see how this approach works for our data. (We have a breakdown of the main ensembling techniques in our full course on algorithms). These algorithms leverage multiple decision trees to either vote or give pass information on to subsequent models. 

In [29]:
from sklearn.ensemble import RandomForestClassifier

#first let's try a non-tuned implementation 
rf = RandomForestClassifier(random_state=42)

rf_accuracy = cross_val_score(rf,X_train,y_train.values.ravel(), cv=3, scoring ='accuracy')
rf_f1 = cross_val_score(rf,X_train,y_train.values.ravel(), cv=3, scoring ='f1')

In [30]:
print('rf_accuracy: ' +str(rf_accuracy))
print('rf F1_Macro Score: '+str(rf_f1))
print('rf_accuracy_avg: ' + str(rf_accuracy.mean()) +'  |  rf_f1_avg: '+str(rf_f1.mean()))

#of course, you can tune this model like the others! 

rf_accuracy: [0.85296764 0.85583357 0.85314591]
rf F1_Macro Score: [0.19750813 0.20154459 0.20347667]
rf_accuracy_avg: 0.8539823710331328  |  rf_f1_avg: 0.20084313198813952


In [31]:
print([int(x) for x in np.linspace(10, 110, num = 11)]+[None])#.append(None))

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None]


In [32]:
random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)]+[None],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}
rs_rf = RandomizedSearchCV(estimator = rf, param_distributions =random_grid, n_iter =100, cv = 3, random_state = 42, scoring ='f1')

rs_rf.fit(X_train,y_train.values.ravel())

KeyboardInterrupt: 

In [33]:
from sklearn.metrics import f1_score

nb.fit(X_train,y_train.values.ravel())
ens.fit(X_train,y_train.values.ravel())
dt_voting.fit(X_train,y_train.values.ravel())
ens_stack.fit(X_train,y_train.values.ravel())
rf_est = RandomForestClassifier()
rf_est.fit(X_train,y_train.values.ravel())

nb_pred = nb.predict(X_test)
ens_pred = ens.predict(X_test)
dt_pred = dt_voting.predict(X_test)
ens_stack_pred = ens_stack.predict(X_test)
rf_pred = rf_est.predict(X_test)

print('baseline score ' + str(f1_score(y_test,nb_pred)))
print('dt score ' + str(f1_score(y_test,dt_pred)))
print('voting score ' + str(f1_score(y_test,ens_pred)))
print('Stacking score ' + str(f1_score(y_test,ens_stack_pred)))
print('rf score ' + str(f1_score(y_test,rf_pred)))

/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chongbei/Workspace/python/initial-bid/bid-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sci

baseline score 0.41692416439462787
dt score 0.2824647279009502
voting score 0.33137630919014605
Stacking score 0.43129720668286525
rf score 0.20566135815887718


# Summary
In this notebook we implemented the following: 
We will cover:
- Baseline creation
- Model selection
- Parameter tuning
     - manual
     - gridsearch
     - random search
     - basian optomization
- Ensemble models
